In [1]:
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sdv.evaluation.single_table import run_diagnostic
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import get_column_plot
from sdv.evaluation.single_table import get_column_pair_plot
from sdv.evaluation.single_table import evaluate_quality
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

os.chdir('/home/onyxia/work/synthetic-data-sdc/')
os.environ["MLFLOW_TRACKING_URI"] = "https://projet-donnees-synthetiques-mlflow.user.lab.sspcloud.fr/" 

In [2]:
variables = ['sex','age','agegr','placesize','edu','socprof','marital','ls','depress','trust','trustfam','trustneigh','sport','nofriend','smoke','alcabuse','alcsol','wkabint','englang','height','weight','bmi']
original = pd.read_csv('SDV/df_original.csv', names = variables)
sample = pd.read_csv('SDV/df1_sample.csv', names = variables)
cart = pd.read_csv('SDV/df1_cart.csv', names = variables)
ctree = pd.read_csv('SDV/df1_ctree.csv', names = variables)
parametric = pd.read_csv('SDV/df1_parametric.csv', names = variables)
rf = pd.read_csv('SDV/df1_rf.csv', names = variables)
bag = pd.read_csv('SDV/df1_bag.csv', names = variables)

In [3]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(original)
metadata

{
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "columns": {
        "sex": {
            "sdtype": "categorical"
        },
        "age": {
            "sdtype": "numerical"
        },
        "agegr": {
            "sdtype": "categorical"
        },
        "placesize": {
            "sdtype": "categorical"
        },
        "edu": {
            "sdtype": "categorical"
        },
        "socprof": {
            "sdtype": "categorical"
        },
        "marital": {
            "sdtype": "categorical"
        },
        "ls": {
            "sdtype": "categorical"
        },
        "depress": {
            "sdtype": "numerical"
        },
        "trust": {
            "sdtype": "categorical"
        },
        "trustfam": {
            "sdtype": "categorical"
        },
        "trustneigh": {
            "sdtype": "categorical"
        },
        "sport": {
            "sdtype": "categorical"
        },
        "nofriend": {
            "sdtype": "numerical"
        },
   

In [6]:
synthesizer = TVAESynthesizer(metadata, verbose=True)
start_time = time.time()
synthesizer.fit(original)
end_time = time.time()
elapsed_time = end_time - start_time

start_time1 = time.time()
synthetic_data = synthesizer.sample(num_rows=len(original))
end_time1 = time.time()
elapsed_time1 = end_time1 - start_time1



print(f"Temps pris pour entraîner le modèle : {elapsed_time} secondes")
print(f"Temps pris pour générer les données : {elapsed_time1} secondes")

Loss: 16.792: 100%|██████████| 300/300 [21:22<00:00,  4.27s/it] 


Temps pris pour entraîner le modèle : 1296.561350107193 secondes
Temps pris pour générer les données : 0.6361339092254639 secondes


In [8]:
synthetic_data.to_csv("tvae.csv")

## Hyperparamètres

- **epochs** et **batch_size** : ces arguments contrôlent le nombre d'itérations que le modèle effectuera pour optimiser ses paramètres, ainsi que le nombre d'échantillons utilisés à chaque étape. Ses valeurs par défaut sont respectivement 300 et 500, et **batch_size doit toujours être une valeur multiple de 10**. Ces hyperparamètres ont un effet très direct sur la durée du processus de formation, mais également sur les performances des données. Ainsi, pour les nouveaux ensembles de données, vous souhaiterez peut-être commencer par définir une valeur faible sur les deux pour voir combien de temps prend le processus de formation. vos données et plus tard, augmentez le nombre à des valeurs acceptables afin d'améliorer les performances.

- **log_frequency** : s'il faut utiliser la log fréquence des modalités catégorieles dans l'échantillonnage conditionnel. La valeur par défaut est True. Cet argument affecte la manière dont le modèle traite les fréquences des valeurs catégorielles utilisées pour conditionner le reste des valeurs. Dans certains cas, le remplacer par False pourrait entraîner de meilleures performances.

- **embedding_dim (int)** : taille de l'échantillon aléatoire transmis au générateur. La valeur par défaut est 128.

- **compress_dims (tuple ou liste d'entiers)** : taille de chaque couche cachée dans l'encodeur. La valeur par défaut est (128, 128).

- **decompress_dims (tuple ou liste d'entiers)** : Taille de chaque couche cachée dans le décodeur. La valeur par défaut est (128, 128).

- **l2scale (int)** : terme de régularisation. La valeur par défaut est 1e-5.

- **batch_size (int)** : nombre d'échantillons de données à traiter à chaque étape.

- **loss_factor (int)** : Multiplicateur de l'erreur de reconstruction. La valeur par défaut est 2.

- **cuda (bool ou str)** : si True, utilisez CUDA. S'il s'agit d'une str, utilisez l'appareil indiqué. Si False, n’utilisez pas du tout cuda.

In [16]:
with open("SDV/config_tvae.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

class TVAEWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None

    def load_context(self, context):
        self.model = joblib.load(context.artifacts["model_path"])

    def predict(self, context, model_input):
        return self.model.sample(len(model_input))

remote_server_uri = os.environ["MLFLOW_TRACKING_URI"]
experiment_name = 'Default'
run_name = 'tvae_comp_256_256_decomp_256_256'

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name=run_name):
    for param_key, param_value in config.items():
        if param_key != 'mlflow':
            mlflow.log_param(param_key, param_value)
    mlflow.log_param('run_name', run_name)

    #Entraînement
    start_time = time.time()
    tvae = TVAESynthesizer(metadata,
                           verbose=True,
                           compress_dims=[256,256],
                           decompress_dims=[256,256],
                        epochs=10)
    tvae.fit(original)
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    mlflow.log_metric("elapsed_time", elapsed_time)
    print(f"Temps pris entraîner le modèle : {elapsed_time} secondes")
    
    params_tvae = tvae.get_parameters()

    # Log des paramètres du modèle
    for param_key, param_value in params_tvae.items():
        with mlflow.start_run(nested=True):
            mlflow.log_param(param_key, param_value)

    loss = tvae.get_loss_values()
    losses = loss['Loss'].tolist()
    for epoch, loss in enumerate(losses):
        mlflow.log_metric("loss", loss, step=epoch)

    model_path = "tvae_model.pkl"
    joblib.dump(tvae, model_path)
    mlflow.pyfunc.log_model(
        artifact_path="tvae_model",
        python_model=TVAEWrapper(),
        artifacts={"model_path": model_path},
    )
mlflow.end_run()

Loss: 40.776: 100%|██████████| 10/10 [00:21<00:00,  2.16s/it]


Temps pris entraîner le modèle : 34.86447620391846 secondes


In [15]:
tvae.get_parameters()

{'enforce_min_max_values': True,
 'enforce_rounding': True,
 'embedding_dim': 128,
 'compress_dims': (128, 128),
 'decompress_dims': (128, 128),
 'l2scale': 1e-05,
 'batch_size': 500,
 'verbose': True,
 'epochs': 500,
 'loss_factor': 2,
 'cuda': True}

## Grid search

In [ ]:
import itertools
import yaml
from yaml.loader import SafeLoader
import mlflow
from sdv.single_table import TVAESynthesizer
import mlflow.pyfunc
import joblib
import os
import time
import pandas as pd

# Charger les configurations du fichier YAML
with open("SDV/config_tvae.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

class TVAEWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.model = None

    def load_context(self, context):
        self.model = joblib.load(context.artifacts["model_path"])

    def predict(self, context, model_input):
        return self.model.sample(len(model_input))

remote_server_uri = os.environ["MLFLOW_TRACKING_URI"]
experiment_name = 'Default'
run_name = 'grid_search'

mlflow.set_tracking_uri(remote_server_uri)
mlflow.set_experiment(experiment_name)

compress_dims_values = config['compress_dims']
decompress_dims_values = config['decompress_dims']

param_combinations = list(itertools.product(compress_dims_values, decompress_dims_values))

for param_set in param_combinations:
    compress_dims, decompress_dims = param_set
    run_name = f"tvae_compress_dims_{compress_dims}_decompress_dims_{decompress_dims}"

    with mlflow.start_run(run_name=run_name):
        # Log des paramètres de configuration
        mlflow.log_param('compress_dims', compress_dims)
        mlflow.log_param('decompress_dims', decompress_dims)
        
        # Entraînement
        start_time = time.time()
        tvae = TVAESynthesizer(metadata,
                           verbose=True,
                           compress_dims=compress_dims,
                           decompress_dims=decompress_dims)
        tvae.fit(original)
        end_time = time.time()

        elapsed_time = end_time - start_time
        mlflow.log_metric("elapsed_time", elapsed_time)
        print(f"Temps pris pour entraîner le modèle avec epochs={epochs}: {elapsed_time} secondes")

        # Log des pertes du générateur et du discriminateur
        loss = tvae.get_loss_values()
        generator_losses = loss['Generator Loss'].tolist()
        discriminator_losses = loss['Discriminator Loss'].tolist()
        for epoch, loss in enumerate(generator_losses):
            mlflow.log_metric("generator_loss", loss, step=epoch)
        for epoch, loss in enumerate(discriminator_losses):
            mlflow.log_metric("discriminator_loss", loss, step=epoch)

        # Sauvegarde du modèle
        model_path = f"tvae_model_{run_name}.pkl"
        joblib.dump(tvae, model_path)
        mlflow.pyfunc.log_model(
            artifact_path="tvae_model",
            python_model=TVAEWrapper(),
            artifacts={"model_path": model_path},
        )
    mlflow.end_run()

Loss: 32.918:  30%|███       | 90/300 [02:55<05:00,  1.43s/it]